## Importar Librerias

In [2]:
# modulos de python
import ast

# modulos de terceros
import pandas as pd



> Importacion solo de las librerias a utilizar durante el ETL de los datos.

## Cargar datos.

In [3]:
# cargar datos de los creditos
path = './data/credits.csv'
data_cd = pd.read_csv(path)
data_cd

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [ ]:
data_cd['cast'][1]

> carga de los datos de 'credits' y se evalua su informacion para poder proceder con las transformaciones.

In [ ]:
# Evaluar los datos de todas las columnas.
data_cd.info()

## Cargar los datos de 'movies'

In [ ]:
# Cargar los datos de las Peliculas.
path = './data/movies_dataset.csv'
data_movies = pd.read_csv(path, low_memory=False)
data_movies

> Una vez cargado los datos se realiza una evaluacion e interpretacion para proceder con la limpueza y estructuracion del conjunto de datos (ETL)

## Transformar columna 'id'

In [ ]:
# Transformar los datos de tipo int a str
data_cd['id'] = data_cd['id'].astype(str)

> Se tomo la decision de hacer un merge de ambos data sets con el fin de facilitar navegacion relacional de los datos entre las columnas, para ello era requerida la transformacion de la columna 'id' en los datos de 'credits' para poder manejar el mismo tipo de datos en ambas columnas y pueda hacerse un merge de ambos dataframes.

## Combinar Dataframes

In [ ]:
# combinar datasets
data = pd.merge(data_movies, data_cd, on='id', how='inner')

In [ ]:
# Verificar data combinada
data

In [ ]:
data.isnull().sum()

## Extraer datos de los jsons anidados

In [ ]:
# Definir una función para extraer los datos anidados en listas.
def extract_list_str(list_str):
    '''la funcion realiza un casteo de str dependiendo
    de la sintaxis al tipo de estructura que se reconozca 
    para manipularse y retornar datos de la pelicula como str
      '''
    # Manejo de errores.
    try:
        # condicion para verificar que la el contexto del str no sea para una lista vacia.
        if list_str != '[]':

            # Usar el modulo ast para castear la el tipo a la estructura deseada.
            list_ = ast.literal_eval(list_str)
            # unir valores de la lista de js mediante lista comprimida y join.
            return ', '.join([dic['name'] for dic in list_])
        else:
            # Retornar NoData si es una lista vacia.
            return 'NoData'
    except:
        # Retornar NoData si es un valor nulo o existe otro tipo de error.
        return 'NoData'


# Definir una función para extraer los datos anidados de .
def extract_json_str(collection):
    '''la funcion realiza un casteo de str dependiendo
    de la sintaxis al tipo de estructura que se reconozca 
    para manipularse y retornar la coleccion de la pelicula como str
      '''
    # Manejo de errores.
    try:
        # Usar el modulo ast para castear la el tipo a la estructura deseada.
        collection_dict = ast.literal_eval(collection)
        # Devolver el valor extraido del diccionario.
        return collection_dict['name']
    except:
        # Retornar NoData si es un valor nulo o existe otro tipo de error.
        return 'NoData'

# Definir una función para extraer los datos anidados de genres.
def extract_director(director):
    '''la funcion realiza un casteo de str dependiendo
    de la sintaxis al tipo de estructura que se reconozca 
    para manipularse y retornar datos de la pelicula como str
      '''
    # Manejo de errores.
    try:
        # condicion para verificar que la el contexto del str no sea para una lista vacia.
        if director != '[]':
            # Usar el modulo ast para castear la el tipo a la estructura deseada.
            list_ = ast.literal_eval(director)
            # For para iterar la lista y encontrar el director entre el crew.
            for i in list_:
                if i['job'] == 'Director':
                    return i['name']
                else:
                    continue
        else:
            # Retornar NoData si es una lista vacia.
            return 'NoData'
    except:
        # Retornar NoData si es un valor nulo o existe otro tipo de error.
        return 'NoData'

# Aplicar las funciones de extracción a las columnas correspondientes
data['genres_clean'] = data['genres'].apply(extract_list_str)
data['collection_clean'] = data['belongs_to_collection'].apply(extract_json_str)
data['production_companies_clean'] = data['production_companies'].apply(extract_list_str)
data['production_countries_clean'] = data['production_countries'].apply(extract_list_str)
data['spoken_languages_clean'] = data['spoken_languages'].apply(extract_list_str)
data['cast_clean'] = data['cast'].apply(extract_list_str)
data['crew_clean'] = data['crew'].apply(extract_director)
# Mostrar las primeras filas del dataframe con las nuevas columnas
data

de errores mediante try an except
        en donde si no hay errores el bloque se realizara una validacion mediante un if
        que validara que no tenga el str un formato de lista vacia, no ser asi
        mediante el modulo ast que analiza la sintaxis
        de un string se hace un casteo automatico segun el texto, 
        para luego poder demanera mas sencilla manipular el dato como una lista
        iterandolo mediante una lista por compresion y a su vez el metodo join va agregando
        cada elemento como un string separados por una coma y un espacio (', ') que al final sera
          retornado 

## Eliminar columnas

In [ ]:
drop_colums = ['video','poster_path', 'homepage','imdb_id','adult','poster_path',
               'tagline','original_language', 'belongs_to_collection','genres',
               'production_companies','production_countries','spoken_languages',
                'cast', 'crew'
            ]
data = data.drop(columns=drop_colums)
data

In [ ]:
data.info()

In [ ]:
data.isnull().sum().sort_values(ascending=False)

## Llenar valores nulos de 'revenue' y 'budget'

In [ ]:
data['revenue'] = data['revenue'].fillna(0)
data['revenue']

In [ ]:
data['revenue'][data['revenue'].isnull()]

In [ ]:
data['budget'] = data['budget'].astype('float64')
type(data['budget'][1])

## Eliminar valores nulos de 'release_date'

In [ ]:
data= data.dropna(subset=['release_date'])
data

In [ ]:
data

In [ ]:
data['release_date'][45461]

In [ ]:
data.isnull().sum().sort_values(ascending=False)

## Normalizar fecha

In [ ]:
data['release_date']

In [ ]:
data.release_date = pd.to_datetime(data['release_date'], format='%Y-%m-%d', errors='coerce')
data.loc[:, 'release_date'] = pd.to_datetime(data['release_date'], format='%Y-%m-%d', errors='coerce')
data['release_date']

## Crear columna 'release_year'

In [ ]:
data.loc[:, 'release_year'] = data['release_date'].dt.year

In [ ]:
data['release_year']

## Crear columna 'retorno_de_inversion'

In [ ]:
#data.loc[:, 'retorno_de_inversion'] = data.apply(lambda row: row['revenue']/row['budget'] if row['budget'] !=0 else 0, axis=1)
data.loc[:, 'retorno_de_inversion'] = data.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)
data['retorno_de_inversion']

In [ ]:
data